In [8]:
# Install required libraries
# Run these commands in your terminal or command prompt:
# pip install yfinance talib scikit-learn numpy pandas

# Import libraries
import yfinance as yf
import talib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Fetch historical data for Apple (AAPL)
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2023-01-01")

# Convert the 'Close' column to a NumPy array
close_prices = data['Close'].values
print(f"Shape of close_prices: {close_prices.shape}")

# Flatten the array (though it's already 1D, this ensures it)
close_prices = close_prices.flatten()
print(f"Shape of close_prices after flattening: {close_prices.shape}")

# Calculate technical indicators using TA-Lib
data['SMA'] = talib.SMA(close_prices, timeperiod=30)  # 30-day SMA
data['RSI'] = talib.RSI(close_prices, timeperiod=14)  # 14-day RSI
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(close_prices)  # MACD

# Drop rows with missing values (due to rolling window calculations)
data.dropna(inplace=True)

# Normalize the features (SMA and RSI)
scaler = MinMaxScaler()
data[['SMA', 'RSI']] = scaler.fit_transform(data[['SMA', 'RSI']])

# Prepare features (X) and target (y)
X = data[['SMA', 'RSI']]  # Features: SMA and RSI
y = data['Close']         # Target: Close price

# Split the data chronologically (train on first 80%, test on last 20%)
train_size = int(len(data) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Initialize the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Display results
print("\nFirst few rows with indicators:")
print(data[['Close', 'SMA', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist']].head())
print(f"\nRMSE: {rmse}")
print(f"R-squared: {r2}")

[*********************100%***********************]  1 of 1 completed

Shape of close_prices: (756, 1)
Shape of close_prices after flattening: (756,)

First few rows with indicators:
Price           Close       SMA       RSI      MACD MACD_signal MACD_hist
Ticker           AAPL                                                    
Date                                                                     
2020-02-20  77.730087  0.129498  0.483962  0.955805    1.123804 -0.167999
2020-02-21  75.970680  0.130282  0.360644  0.729543    1.044951 -0.315409
2020-02-24  72.362030  0.129487  0.172179  0.256088    0.887179 -0.631091
2020-02-25  69.910950  0.127892  0.078148 -0.313298    0.647083 -0.960382
2020-02-26  71.019989  0.126144  0.155312 -0.667358    0.384195 -1.051553

RMSE: 2.7440714331833242
R-squared: 0.9341269843368437
